# MIDITok - Tokenizing, Training, Generating!

## 1. Install & Import necessary libraries

In [1]:
!pip install datasets transformers -q
!pip install miditok -q
!pip install symusic -q

In [2]:
from miditok import MMM, MuMIDI, TokenizerConfig
from miditok.pytorch_data import DatasetTok, DataCollator
from pathlib import Path
from symusic import Score

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Tokenizing

MidiTok 에서 제공하는 tokenizer 클래스로 tokenizer 생성

In [3]:
# Creating a multitrack tokenizer configuration, read the doc to explore other parameters

config = TokenizerConfig(
    num_velocities=16, 
    use_chords=True, 
    use_programs=True,
    use_pitch_intervals=True
    )

TOKENIZER_NAME = MMM # MMM 토크나이저 사용
# TOKENIZER_NAME = MuMIDI # MuMIDI 토크나이저 사용
tokenizer = TOKENIZER_NAME(config)

미디 파일 하나를 시험삼아 토크나이징 해봅니다

In [4]:
# Loads a midi, converts to tokens, and back to a MIDI

midi = Score("data/jazz-chunked-16bars/003_20thcenturystomp_cleaned/1.mid")
tokens = tokenizer(midi)  # calling the tokenizer will automatically detect MIDIs, paths and tokens
converted_back_midi = tokenizer(tokens)  # PyTorch / Tensorflow / Numpy tensors supported

토크나이징 결과 확인

In [6]:
len(tokens)
# tokens

1569

토큰에서 다시 생성된 미디 파일 정보 확인

In [7]:
converted_back_midi

Score(ttype=Tick, tpq=8, begin=0, end=596, tracks=4, notes=402, time_sig=1, key_sig=0, markers=0, lyrics=0)

학습할 미디 파일들의 경로 지정

In [8]:
midi_paths = list(Path("data/jazz-chunked-16bars").glob("**/*.mid"))
len(midi_paths)

1828

## 3. Train / Validation Dataset 구축

train / valid 데이터셋 split 및 shuffle

In [9]:
# Split MIDI paths in train/valid/test sets

from random import shuffle

total_num_files = len(midi_paths)
num_files_valid = round(total_num_files * 0.1) # Validation 비율 자유롭게 변경
shuffle(midi_paths)
midi_paths_valid = midi_paths[:num_files_valid]
midi_paths_train = midi_paths[num_files_valid:]

##### ⏰ 데이터를 MidiTok Dataset 형태로 처리 (V100 서버 기준 재즈 데이터에 약 1분 소요)

In [10]:
# Creates a Dataset and a collator to be used with a PyTorch DataLoader to train a model
dataset_train = DatasetTok(
    files_paths=midi_paths_train,
    min_seq_len=50,
    max_seq_len=2046,
    tokenizer=tokenizer,
)
dataset_valid = DatasetTok(
    files_paths=midi_paths_valid,
    min_seq_len=50,
    max_seq_len=2046,
    tokenizer=tokenizer,
)
collator = DataCollator(
    tokenizer["PAD_None"], tokenizer["BOS_None"], tokenizer["EOS_None"], copy_inputs_as_labels=True
)

Loading data: data/jazz-chunked-16bars/193_LibertyCity_cleaned: 100%|██████████| 1645/1645 [00:34<00:00, 47.76it/s]
Loading data: data/jazz-chunked-16bars/288_shawnuff_cleaned: 100%|██████████| 183/183 [00:03<00:00, 47.71it/s]


## 4. HuggingFace와 호환 가능한 형태로 데이터셋 변경
❓ HuggingFace Trainer로 train 시키기 위해 아래 과정을 수행합니다.<br>
❗️ 아래 셀들이 조금 필요없는 작업일 수 있는데, 일단 구현을 우선으로 작성 해놓았습니다. 혹시 불필요한 과정 발견하시면 제보 부탁드립니다!

In [11]:
from torch.utils.data import DataLoader

data_loader_train = DataLoader(dataset=dataset_train, collate_fn=collator)
data_loader_valid = DataLoader(dataset=dataset_valid, collate_fn=collator)
train_tokenized_songs = []
valid_tokenized_songs = []
for batch in data_loader_train:
    train_tokenized_songs.append(batch)
for batch in data_loader_valid:
    valid_tokenized_songs.append(batch)

In [12]:
len(valid_tokenized_songs[0]['input_ids'][0])

1024

In [14]:
# make custom dataset
import torch
from torch.utils.data import Dataset, DataLoader

class MidiDataset(Dataset):
    def __init__(self, tokenized_songs, max_length=1022):  # max_length를 512로 하면 앞, 뒤에 BOS, EOS 토큰이 또 붙어서 길이 514 되고 에러가 나서 일단 510로 함. 디버깅 필요!!
        self.tokenized_songs = tokenized_songs
        self.max_length = max_length
    
    def __len__(self):
        return len(self.tokenized_songs)
    
    def __getitem__(self, idx):
        # item = {key: val.clone().detach() for key, val in self.tokenized_songs[idx].items()}
        item = {'input_ids': self.tokenized_songs[idx]['input_ids'][:, :self.max_length].clone().detach().squeeze(),}
        return item

In [15]:
train_dataset = MidiDataset(train_tokenized_songs)
eval_dataset = MidiDataset(valid_tokenized_songs)

In [16]:
train_dataset[0]['input_ids'].shape
# train_dataset[0]

torch.Size([1022])

In [17]:
# Test our data_collator
out = collator([train_dataset[i] for i in range(5)])

for key in out:
    print(f"{key} shape: {out[key].shape}")

print(f"out {out}")

input_ids shape: torch.Size([5, 1024])
labels shape: torch.Size([5, 1024])
attention_mask shape: torch.Size([5, 1024])
out {'input_ids': tensor([[  1,   1,   4,  ..., 269, 106,   2],
        [  1,   1, 182,  ...,   0,   0,   0],
        [  1,   1,   4,  ..., 109, 116,   2],
        [  1,   1, 267,  ...,   0,   0,   0],
        [  1,   1,   4,  ..., 116, 304,   2]]), 'labels': tensor([[   1,    1,    4,  ...,  269,  106,    2],
        [   1,    1,  182,  ..., -100, -100, -100],
        [   1,    1,    4,  ...,  109,  116,    2],
        [   1,    1,  267,  ..., -100, -100, -100],
        [   1,    1,    4,  ...,  116,  304,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}


## 4. Training

Custom Trainer 정의

In [18]:
from transformers import Trainer, TrainingArguments

# first create a custom trainer to log prediction distribution
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def evaluation_loop(
        self,
        dataloader,
        description,
        prediction_loss_only=None,
        ignore_keys=None,
        metric_key_prefix="eval",
    ):
        # call super class method to get the eval outputs
        eval_output = super().evaluation_loop(
            dataloader,
            description,
            prediction_loss_only,
            ignore_keys,
            metric_key_prefix,
        )

        return eval_output

세부 training config 정하고, LLaMA-2 모델 불러오기

In [19]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaConfig

# Remove the extra comma at the end of model_name
model_name = "meta-llama/Llama-2-7b-hf"
# model_name = "mistralai/Mistral-7B-v0.1"
model = LlamaForCausalLM.from_pretrained(model_name)

# Display the model configuration
model.config

Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.25s/it]


LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.37.2",
  "use_cache": true,
  "vocab_size": 32000
}

New config


In [18]:
# Make a copy of the original config
new_config = LlamaConfig(
    hidden_size = 2048,
    intermediate_size = 4096,
    num_hidden_layers = 8,
    num_attention_heads = 8,
    vocab_size = len(tokenizer),
    torch_dtype = "float32",
    pad_token_id = tokenizer['PAD_None'],
    bos_token_id = tokenizer['BOS_None'],
    eos_token_id = tokenizer['EOS_None']
)

In [19]:
downsized_model = LlamaForCausalLM(config=new_config)

In [20]:
import os
os.environ["WANDB_PROJECT"] = "MusicAI"  # name your W&B project
wandb_name = "llama2-16bars-test"
# wandb_name = "mistral-7b-16bars-test"

Training Argument 설정

In [21]:
# Create the args for out trainer
from argparse import Namespace

# Get the output directory with timestamp.
output_path = "models"
steps = 100
# Commented parameters correspond to the small model
config = {"output_dir": output_path,
          "num_train_epochs": 2, # 학습 epoch 자유롭게 변경. 저는 30 epoch 걸어놓고 early stopping 했습니다.
          "per_device_train_batch_size": 4,
          "per_device_eval_batch_size": 2,
          "evaluation_strategy": "steps",
          "save_strategy": "steps",
          "eval_steps": steps,
          "logging_steps":steps,
          "logging_first_step": True,
          "save_total_limit": 5,
          "save_steps": steps,
          "lr_scheduler_type": "cosine",
          "learning_rate":5e-4,
          "warmup_ratio": 0.01,
          "weight_decay": 0.01,
          # "seed": 1, # randomize 해보기
          "load_best_model_at_end": True,
          # "metric_for_best_model": "eval_loss" # best model 기준 바꾸고 싶을 경우 이 부분 변경 (default가 eval_loss임)
          "report_to": "wandb",
          "run_name": wandb_name
          }

args = Namespace(**config)

In [22]:
from transformers import set_seed
# set_seed(args.seed)
set_seed(42)

In [23]:
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

# mps device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
downsized_model.to(device)

train_args = TrainingArguments(**config)

trainer = CustomTrainer(
    model=downsized_model,
    tokenizer=tokenizer,
    args=train_args,
    data_collator=collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)] # Early Stopping patience 자유롭게 변경
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


#### 드디어! 학습을 시작하세요! 
- V100 서버 기준 3 epoch 약 7분 소요
- 저는 30 epoch 정도 걸어놓고 Early Stopping 했습니다.
- 학습 완료된 후 모델 체크포인트는 models 폴더 안에 저장됩니다.

In [24]:
# Train the model.
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: patcasso21 (patcasso). Use `wandb login --relogin` to force relogin


TrainOutput(global_step=1918, training_loss=1.9037819328845107, metrics={'train_runtime': 2305.6827, 'train_samples_per_second': 3.326, 'train_steps_per_second': 0.832, 'total_flos': 1.5768367641427968e+16, 'train_loss': 1.9037819328845107, 'epoch': 2.0})

## 5. GENERATE

처음 입력할 토큰을 지정해주고, tensor로 바꾸어 generated_ids 변수에 할당합니다

In [41]:
initial_token = "BOS_None" # 시작 토큰을 BOS로 설정해줍니다.

In [42]:
generated_ids = torch.tensor([[tokenizer[initial_token]]])
generated_ids

tensor([[1]])

- iteration number와 현재 시간을 초기화합니다. 
- ts 변수에 저장된 시간 정보는 지금은 안 쓰이고 있는데, 파일명 등에 사용하시면 나중에 모니터링 하실 때 좋습니다.

In [43]:
# Timecode 및 iteration number 초기화
import datetime

iteration_number = 0
ts = datetime.datetime.now().strftime("%y%m%d%H%M%S")

- 아래 셀을 실행하여 생성하세요.
- 여러 번 실행하면 실행 할 때마다 트랙이 추가됩니다.

In [48]:
# Iteration 몇 번 돌았는지 기록
iteration_number += 1
print(f"current iteration : {iteration_number}")

# Encode the conditioning tokens.
input_ids = generated_ids.cuda() # 로컬에서 실행할 때는 cuda() 없애주기

# Generate more tokens.
eos_token_id = tokenizer["Track_End"] # "Track_End" 토큰이 나올 때까지 생성 => iteration당 악기 한 트랙씩 생성하는 원리
temperature = 0.8 # Temperature를 높이면 생성 결과가 더욱 randomize 되는 것 같습니다.
generated_ids = downsized_model.generate(
    input_ids,
    max_length=1024,
    do_sample=True,
    temperature=temperature,
    eos_token_id=eos_token_id,
).cpu()

print(generated_ids)

current iteration : 3
tensor([[  1,   1,   4,  ..., 111, 116, 344]])


생성된 토큰 미디 데이터로 변환

In [49]:
midi = tokenizer.tokens_to_midi(generated_ids[0])
midi

Score(ttype=Tick, tpq=8, begin=0, end=1770, tracks=2, notes=228, time_sig=1, key_sig=0, markers=0, lyrics=0)

변환된 미디 데이터로 test_gen.mid 파일 생성

In [50]:
midi.dump_midi(f'./test_gen_iter_{iteration_number}.mid')

### 🎉 축하합니다!

- 학습 및 생성을 완료하였습니다. 여러 세팅으로 학습해보시고, 생성된 미디 파일을 다운받아 https://bandlab.com/ 등에서 실행시켜보세요
- 서버에서 미디 재생이 안 되기 때문에, 생성된 모델과 miditok-gen.ipynb 파일을 다운받아 로컬에서 생성 실험을 계속하시는 것을 추천드립니다!